### dependency 설치

In [23]:
packages = """
aiohappyeyeballs==2.4.0
aiohttp==3.10.5
aiosignal==1.3.1
annotated-types==0.7.0
anyio==4.4.0
asttokens==2.4.1
attrs==24.2.0
beautifulsoup4==4.12.3
cachetools==5.5.0
certifi==2024.7.4
charset-normalizer==3.3.2
click==8.1.7
colorama==0.4.6
comm==0.2.2
dataclasses-json==0.6.7
debugpy==1.8.5
decorator==5.1.1
Deprecated==1.2.14
dirtyjson==1.0.8
distro==1.9.0
executing==2.0.1
filelock==3.15.4
frozenlist==1.4.1
fsspec==2024.6.1
google-ai-generativelanguage==0.6.4
google-api-core==2.19.1
google-api-python-client==2.142.0
google-auth==2.34.0
google-auth-httplib2==0.2.0
google-generativeai==0.5.4
googleapis-common-protos==1.63.2
greenlet==3.0.3
grpcio==1.65.5
grpcio-status==1.62.3
h11==0.14.0
httpcore==1.0.5
httplib2==0.22.0
httpx==0.27.0
huggingface-hub==0.24.6
idna==3.7
InstructorEmbedding==1.0.1
ipykernel==6.29.5
ipython==8.26.0
jedi==0.19.1
Jinja2==3.1.4
jiter==0.5.0
joblib==1.4.2
jupyter_client==8.6.2
jupyter_core==5.7.2
llama-cloud==0.0.13
llama-index==0.10.67.post1
llama-index-agent-openai==0.2.9
llama-index-cli==0.1.13
llama-index-core==0.10.67
llama-index-embeddings-huggingface==0.2.3
llama-index-embeddings-instructor==0.1.3
llama-index-embeddings-openai==0.1.11
llama-index-indices-managed-llama-cloud==0.2.7
llama-index-legacy==0.9.48.post3
llama-index-llms-gemini==0.2.0
llama-index-llms-openai==0.1.30
llama-index-multi-modal-llms-openai==0.1.9
llama-index-program-openai==0.1.7
llama-index-question-gen-openai==0.1.3
llama-index-readers-file==0.1.33
llama-index-readers-llama-parse==0.1.6
llama-parse==0.4.9
MarkupSafe==2.1.5
marshmallow==3.22.0
matplotlib-inline==0.1.7
minijinja==2.0.1
mpmath==1.3.0
multidict==6.0.5
mypy-extensions==1.0.0
nest-asyncio==1.6.0
networkx==3.3
nltk==3.9.1
numpy==1.26.4
openai==1.42.0
packaging==24.1
pandas==2.2.2
parso==0.8.4
pillow==10.4.0
platformdirs==4.2.2
prompt_toolkit==3.0.47
proto-plus==1.24.0
protobuf==4.25.4
psutil==6.0.0
pure_eval==0.2.3
pyasn1==0.6.0
pyasn1_modules==0.4.0
pydantic==2.8.2
pydantic_core==2.20.1
Pygments==2.18.0
pyparsing==3.1.2
pypdf==4.3.1
python-dateutil==2.9.0.post0
pytz==2024.1
pywin32==306
PyYAML==6.0.2
pyzmq==26.1.1
regex==2024.7.24
requests==2.32.3
rsa==4.9
safetensors==0.4.4
scikit-learn==1.5.1
scipy==1.14.1
sentence-transformers==2.7.0
six==1.16.0
sniffio==1.3.1
soupsieve==2.6
SQLAlchemy==2.0.32
stack-data==0.6.3
striprtf==0.0.26
sympy==1.13.2
tenacity==8.5.0
threadpoolctl==3.5.0
tiktoken==0.7.0
tokenizers==0.19.1
torch==2.4.0
tornado==6.4.1
tqdm==4.66.5
traitlets==5.14.3
transformers==4.44.1
typing-inspect==0.9.0
typing_extensions==4.12.2
tzdata==2024.1
uritemplate==4.1.1
urllib3==2.2.2
wcwidth==0.2.13
wrapt==1.16.0
yarl==1.9.4
"""

with open("requirements.txt", "w") as f:
    f.write(packages)


In [15]:
%pip install -r requirements.txt

  Using cached llama_cloud-0.0.13-py3-none-any.whl.metadata (751 bytes)
  Using cached llama_index-0.10.67.post1-py3-none-any.whl.metadata (11 kB)
  Using cached llama_index_core-0.10.67-py3-none-any.whl.metadata (2.4 kB)
  Using cached llama_index_embeddings_huggingface-0.2.3-py3-none-any.whl.metadata (769 bytes)
  Using cached llama_index_embeddings_instructor-0.1.3-py3-none-any.whl.metadata (810 bytes)
  Using cached llama_index_llms_gemini-0.2.0-py3-none-any.whl.metadata (734 bytes)
  Using cached llama_index_llms_openai-0.1.30-py3-none-any.whl.metadata (650 bytes)
Using cached llama_cloud-0.0.13-py3-none-any.whl (169 kB)
Using cached llama_index-0.10.67.post1-py3-none-any.whl (6.9 kB)
Using cached llama_index_core-0.10.67-py3-none-any.whl (15.5 MB)
Using cached llama_index_embeddings_huggingface-0.2.3-py3-none-any.whl (8.6 kB)
Using cached llama_index_embeddings_instructor-0.1.3-py3-none-any.whl (3.6 kB)
Using cached llama_index_llms_gemini-0.2.0-py3-none-any.whl (5.2 kB)
Using ca

In [1]:
!pip install pytesseract

# llama index를 사용해 gemini api 불러오기

### png 불러오기

In [3]:
from IPython.display import display, Markdown
from llama_index.core import Document
from PIL import Image
import os
import pytesseract

# 문서의 형식 = 텍스트 
# tesseract 를 사용하기 위해서는 따로 설치를 하여야 합니다 
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

image_path = r'data_png\dwld.png'
doc2_text = pytesseract.image_to_string(Image.open(image_path), lang='kor')

# 객체로 변환하기 
doc2 = Document(text=doc2_text)
# Document 객체 출력
print(doc2)

Doc ID: a010d53a-c6d2-44e0-acfc-c508e74e1340
Text: 4 -해상법 중요판례집([) 2012-2017  본 법원은 무모한 행위로 인한 경우에 선급협회가 불법행위책임을
부담해: 전 판단을 하지않고, 다만 그 전제가 되는 무모한 행위가 있는지를 먼저 판단한다, 그러한 무모한 행위가 있었다는
충분한 증거를 제출하지 못하였다. 그러므로 1  에서 내린, 피고가 승소한 약식판결을 재확인한다.  3의견
; 본 사건은 선급협회(025506000ㅁ 50060)의 제3자에 대한 불법행위책임 사안이었다. 선급협회는 선박의 안전성에
대한 검사를 한 다음 선급에 선박을 고 증서를 발행하여주는 기관이다. 이 중서...


In [4]:
from llama_index.core import __version__

print(__version__)

0.10.67


### pdf 불러오기

In [6]:
from llama_index.core import SimpleDirectoryReader

input_dir = r"data_pdf"
reader = SimpleDirectoryReader(input_dir=input_dir)
doc1 = reader.load_data()

# doc1 = SimpleDirectoryReader(input_files=["data"]).load_data()

### 파일 결합

In [7]:
all_docs = all_docs = doc1 + [doc2] 
print(all_docs)

[Document(id_='9b577f8f-cad3-4566-996b-fa23cefa0627', embedding=None, metadata={'page_label': '1', 'file_name': '해운법.pdf', 'file_path': 'c:\\Users\\max99\\OneDrive\\바탕 화면\\aaaa\\haemulhaemul\\data_pdf\\해운법.pdf', 'file_type': 'application/pdf', 'file_size': 244684, 'creation_date': '2024-08-23', 'last_modified_date': '2024-08-23'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='법제처                                                            1                                                       국가법령정보센터\n해운법 \n \n해운법 \n[시행 2024. 5. 1.] [법률 제19807호, 2023. 10. 31., 타법개정] \n해양수산부 (해운정책과) 044-200-5712, 5711 \n해양수산부 (연안해운과) 044-200-5731 \n       제1장 총칙\n \n제1조(목적) 이 법은 해상운송의 질서를 유지하고 공정한 경쟁이 이루어지도록 하며, 해운업의 건전한 발전과 여객ㆍ화\n물의 원활하고 안전한 운송을 도모함으로써 이용자의 편의를 향상시키고 

### 임베딩 다운로드

In [8]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model_ko = HuggingFaceEmbedding(model_name="bespin-global/klue-sroberta-base-continue-learning-by-mnr") 

c:\Users\max99\OneDrive\바탕 화면\aaaa\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\max99\OneDrive\바탕 화면\aaaa\myenv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### llama index 설정

In [9]:
from llama_index.core import VectorStoreIndex
from llama_index.llms.gemini import Gemini
from llama_index.core import ServiceContext

llm = Gemini(model_name='models/gemini-1.5-flash', request_timeout=120.0)

service_context = ServiceContext.from_defaults(llm=llm, chunk_size=800, chunk_overlap=20, embed_model=embed_model_ko)
index = VectorStoreIndex.from_documents(doc1,service_context=service_context,show_progress=True)

index.storage_context.persist()

query_engine = index.as_query_engine()


C:\Users\max99\AppData\Local\Temp\ipykernel_36352\1076867130.py:7: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm, chunk_size=800, chunk_overlap=20, embed_model=embed_model_ko)
Generating embeddings: 100%|██████████| 100/100 [04:22<00:00,  2.63s/it]


### 질문과 답변 출력

In [10]:
response = query_engine.query("항만에 대해 정확한 정의를 설명해봐")

print(response)

response = query_engine.query("해운업에 대한 정의를 말해봐")

print(response)

해운법에 항만에 대한 정의는 명시되어 있지 않습니다. 

해운업은 해상여객운송사업, 해상화물운송사업, 해운중개업, 해운대리점업, 선박대여업 및 선박관리업을 말합니다. 



### 멀티 턴 질문 과 답변

In [11]:
# 멀티 턴 대화를 위한 history 리스트
history = []

def ask_query(query, history):
    # 대화 히스토리에 현재 쿼리를 추가
    history.append({"role": "user", "content": query})
    
    # 대화 히스토리를 문자열로 병합
    formatted_history = "\n".join([f"{item['role']}: {item['content']}" for item in history])
    
    # 쿼리 엔진에 현재 히스토리를 전달하여 응답 생성
    response = query_engine.query(formatted_history)
    
    # 응답을 히스토리에 추가
    history.append({"role": "model", "content": response})
    
    return response

# 예시 대화
query1 = "해운법에 대해 설명"
response1 = ask_query(query1, history)
print(f"Model: {response1}")

query2 = "더 자세히 알려줘"
response2 = ask_query(query2, history)
print(f"Model: {response2}")

query3 = "칸예 웨스트에 대해 설명해줘"
response3 = ask_query(query3, history)
print(f"Model: {response3}")

User: 해운법에 대해 설명
Model: 해운법은 해상운송의 질서를 유지하고 공정한 경쟁을 이루도록 하며, 해운업의 건전한 발전과 여객 및 화물의 원활하고 안전한 운송을 도모하여 이용자의 편의를 향상시키고 국민경제 발전과 공공복리 증진에 이바지하는 것을 목적으로 합니다. 

User: 더 자세히 알려줘
Model: 해운법은 해상운송의 질서를 유지하고 공정한 경쟁을 이루도록 하며, 해운업의 건전한 발전과 여객 및 화물의 원활하고 안전한 운송을 도모하여 이용자의 편의를 향상시키고 국민경제 발전과 공공복리 증진에 이바지하는 것을 목적으로 합니다. 해운법은 해운업의 종류, 해운업체의 등록 및 허가, 운임 및 운송 계약, 선박의 안전 및 환경 보호, 해운사고의 처리 등 해운업과 관련된 다양한 사항을 규정하고 있습니다. 

User: 칸예 웨스트에 대해 설명해줘
Model: 죄송합니다. 칸예 웨스트에 대한 정보는 제공할 수 없습니다. 저는 해운법에 대한 정보만 제공할 수 있습니다. 

